In [26]:
# basic
import requests
import pandas as pd
import os
import json

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread


# ai core sdk
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.label import Label
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

# image processing
from PIL import Image
import base64
import numpy as np
import cv2

In [27]:
# Get AI Core credentials from json file saved in <directory_path>/secrets folder

try:
    # load keyfile from ../secrets/hana.json
    with open("<directory_path>/secrets/aicore.json") as f:
        aicorecreds = json.load(f)
        print("Found AI Core credentials.")
except FileNotFoundError as e:
    print(f"Failed to load keyfile. Please make sure you have stored the file in a folder called 'secrets'. Error: {e}")

Found AI Core credentials.


In [29]:
# Alternative way to create Connection to AI Core
ai_core_client = AICoreV2Client(
    base_url = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com" + "/v2", # The present SAP AI Core API version is 2
    auth_url=  aicorecreds["url"]  + "/oauth/token", # Suffix to add
    client_id = aicorecreds["clientid"],
    client_secret = aicorecreds["clientsecret"],
    resource_group = "default"
)

In [3]:
# Create Connection to AI Core
ai_core_client = AICoreV2Client(
    base_url = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com" + "/v2", # The present SAP AI Core API version is 2
    auth_url=  "<url>" + "/oauth/token", # Suffix to add
    client_id = "<clientid>,
    client_secret = "<clientsecret>",
    resource_group = "default"
)

In [ ]:
#list all available deployments
deployments = ai_core_client.deployment.query()
for deployment in deployments.resources:
    print(f"{deployment.id}")

In [ ]:
# get the details of a certain deployment
response = ai_core_client.deployment.get(
    deployment_id="d0c5681323fd9f8b",
    resource_group="default"
)

print(response.__dict__)

In [24]:
#Prepare the input for the Image Classification Prediction to be sent to BTP
def images_to_json(image_path, image_name):
    img_np = cv2.imencode('.jpg', cv2.imread(image_path+image_name))
    image_string = base64.b64encode(img_np[1]).decode()
  
    return image_string

In [32]:
# Variables
dir_path = "<>"
deployment_url = '<>'

In [ ]:
# get image prediction labels for all of the images in a directory

image_names = os.listdir(dir_path)
number_of_images = len(image_names)
fig, axes = plt.subplots(1, number_of_images, figsize=(20, 4))

for i in range(number_of_images):
    image_name = image_names[i]
    endpoint = f"{deployment_url}/v2/labelpredict" # endpoint implemented in serving engine
    headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "Content-Type": "application/json"}

    image_string = images_to_json(dir_path, image_name)
    
    predict = requests.post(endpoint, headers=headers, json={'image_name': image_name, 'image_string': image_string})
    
    image_path = os.path.join(dir_path, image_name)
    image = Image.open(image_path)
    axes[i].imshow(image)
    axes[i].set_title(f"The predicted label is: {predict.json().get('label')}")
    axes[i].axis('off')
plt.show()